# Pre-Processing of Titanic's dataframe



On va se servir de l'exemple de *Titanic* pour apprendre à traiter du texte, via la **recherche d'expression régulière** (regexp) de la librairie $re$ .

Le but de ce petit notebook est de vous aider à prendre en main les données de type texte, grace aux fonctions de traitement de textes des packages $re$ et $nltk$. Ainsi, vous saurez transformer les données brutes au format txt en des données exploitables au format pandas.  

Commençons par importer les packages nécessaires au traitement, puis les données.


In [1]:
import pandas as pd
import re

df_titanic = pd.read_csv('../Data/titanic.csv')

df_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Name

On va s'intéresser à la colonne "Name" du dataframe :
    - repérer la structure 
    - extraire : nom, prénon, et titre
    
Regardons-le de plus près :

In [2]:
df_titanic['Name'].head()

0                              Braund, Mr. Owen Harris
1    Cumings, Mrs. John Bradley (Florence Briggs Th...
2                               Heikkinen, Miss. Laina
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                             Allen, Mr. William Henry
Name: Name, dtype: object

## Last name

On remarque que le nom se trouve systématiquement être le **premier mot**, juste avant une **virgule**.  
On va donc rechercher cette virgule (","), couper le texte en morceaux (split), et prendre le premier morceau ([0]).

In [3]:
df_titanic['LastName'] = df_titanic['Name'].apply(lambda x : x.split(",")[0])
df_titanic.LastName.head()

0       Braund
1      Cumings
2    Heikkinen
3     Futrelle
4        Allen
Name: LastName, dtype: object

## Size name

In [4]:
df_titanic['SizeName'] = df_titanic['LastName'].apply( lambda x : len(x))

df_titanic[['LastName','SizeName']].head()

,LastName,SizeName
0,Braund,6
1,Cumings,7
2,Heikkinen,9
3,Futrelle,8
4,Allen,5


# Title

In [5]:
df_titanic['Title'] = df_titanic['Name'].apply( lambda x : x.split(", ")[1]).apply(lambda x : x.split(".")[0])
df_titanic['Title']

0        Mr
1       Mrs
2      Miss
3       Mrs
4        Mr
       ... 
886     Rev
887    Miss
888    Miss
889      Mr
890      Mr
Name: Title, Length: 891, dtype: object

# First name

Allez,  maintenant qu'on a compris comment ça marche, on va essayer d'extraire le prénom :

In [6]:
def extract_name(x):
    if x.find("(") != -1:
        if x.find('("') != -1:
            name = re.search('(?<=\. )[A-Za-z]*', x).group()
        else:
            name = re.search('(?<=\()[A-Za-z]*', x).group()    
    else:
        name = re.search('(?<=\.) [A-Za-z]*', x).group()
    return name

In [7]:
df_titanic['FirstName'] = df_titanic['Name'].apply(lambda x: extract_name(x))
df_titanic['FirstName']

0            Owen
1        Florence
2           Laina
3            Lily
4         William
          ...    
886        Juozas
887      Margaret
888     Catherine
889          Karl
890       Patrick
Name: FirstName, Length: 891, dtype: object

Allez, on regarder si le Titre changait la proba de mourir sur le bateau :

In [ ]:
df_titanic.groupby(['Survived','Title']).size().unstack()

pd.crosstab(df_titanic["Survived"], df_titanic["Title"])